# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## Импорт и изучение данных

**Описание данных**
    
    Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd #импорт библиотеки pandas

from sklearn.model_selection import train_test_split #импорт инструмента для разделения данных
from sklearn.tree import DecisionTreeClassifier #импорт инструмента для формирования модели дерева решений
from sklearn.ensemble import RandomForestClassifier #импорт инструмента для формирвоания модели случайного леса
from sklearn.linear_model import LogisticRegression #импорт модели логистической регрессии

from sklearn.metrics import accuracy_score #импорт метрики accuracy

from tqdm import tqdm #импорт библиотеки для отображения процесса выполнения кода

from sklearn.dummy import DummyClassifier #импорт библиотеки формирования константной модели

In [2]:
random_state=12345

In [3]:
#импорт данных методом read_csv
df=pd.read_csv('data_original/users_behavior.csv')

In [4]:
#вызов информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#вызов первых пяти строк датафрейма
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Из полученной информации можно утверждать, что пропусков в данных нет, соблюдается единый стиль в названиях столбцов, корректный тип данных в столбцах. Предобработка действительно не нужна.

In [6]:
#Рассчитаем коэффициент корреляции Пирсона для различных столбцов относительно столбца с информацией о тарифе.
#Чем ближе к 1 - тем явнее прямая зависимость между данными
for i in df.columns[:4]:
    print(i,df[i].corr(df['is_ultra']))

calls 0.20712155898607415
minutes 0.20695463812169365
messages 0.20382983853553452
mb_used 0.1985676456197413


 Как видно из расчётов, столбцы с информацией о кол-ве звонков, минут, сообщений и использованных мегабайт коррелируют с тарифом незначительно и примерно одинаково.

## Разбиение данных на выборки

In [7]:
#извлечение признаков для модели, выбрасываем столбец is_ultra
features=df.drop(['is_ultra'], axis=1)

#извлечение целевого признака (столбец is_ultra)
target=df['is_ultra']

In [8]:
#разделение данных на тренировочную, тестовую и валидационную выборки в соотношении 60:20:20 инструментом train_test_split
features_train, features_validtest, target_train, target_validtest=\
train_test_split(features,target,train_size=0.6,random_state=random_state, stratify=target)

features_valid, features_test, target_valid, target_test=\
train_test_split(features_validtest, target_validtest, train_size=0.5,random_state=random_state, stratify=target_validtest)

## Исследуование моделей

### Тренировочные выборки

#### Применение модели дерева решений

In [9]:
#обучение модели дерева решений при стандартных параметрах
model_1=DecisionTreeClassifier(random_state=random_state)
model_1.fit(features_train,target_train)
#предсказание для валидационной выборки
model_1_predictions_valid=model_1.predict(features_valid)
#подсчет точности предсказаиий
accuracy_score(model_1_predictions_valid,target_valid)

0.7013996889580093

Обновлённый вывод: Точность предсказаний составляет 0.7 при стандартных гиперпараметрах.  Посмотрим на глубину дерева, при которой достигается данное значение точности:

In [10]:
#Создадим цикл для подсчета точности предсказаний в зависимости от параметра максимальной глубины дерева решений

best_model=None
best_result=0
best_depth=0

for i in range(1,30):
    model_1=DecisionTreeClassifier(random_state=random_state,max_depth=i)
    model_1.fit(features_train,target_train)
    model_1_predictions_valid=model_1.predict(features_valid)
    result=accuracy_score(model_1_predictions_valid,target_valid)
    if result>best_result:
        best_model=model_1
        best_result=result
        best_depth=i
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'для соответствующей глубины:',best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535 для соответствующей глубины: 8


Обновлённый вывод: Точность предсказаний составляет 0.8 при глубине дерева 8.

#### Применение модели случайного леса

In [11]:
#обучение модели случайного леса при стандартных параметрах
model_2=RandomForestClassifier(random_state=random_state)
model_2.fit(features_train,target_train)
#предсказание для валидационной выборки
model_2_predictions_valid=model_2.predict(features_valid)
#подсчет точности предсказаиий
accuracy_score(model_2_predictions_valid,target_valid)

0.80248833592535

Обновлённый вывод: Точность предсказаний составляет 0.8 при стандартных параметрах модели. Посмотрим на параметр n_estimators, при которой достигается данное значение точности:

In [12]:
#Создадим цикл для подсчета точности предсказаний в зависимости от параметра n_estimators

best_model=None
best_result=0
best_estimators=0

for i in tqdm(range(1,100)):
    model_2=RandomForestClassifier(random_state=random_state,n_estimators=i)
    model_2.fit(features_train,target_train)
    model_2_predictions_valid=model_2.predict(features_valid)
    result=accuracy_score(model_2_predictions_valid,target_valid)
    if result>best_result:
        best_model=model_2
        best_result=result
        best_estimators=i
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'при соответствующем n_estimators:',best_estimators)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [01:18<00:00,  1.25it/s]

Accuracy наилучшей модели на валидационной выборке: 0.8118195956454122 при соответствующем n_estimators: 58


Обновлённый вывод: Точность предсказаний увеличилась, но незначительно: 0.81 против 0.8 относительно стандартных гиперпараметров.

#### Применение модели логистической регрессии

In [13]:
#обучение модели логистической регрессии при стандартных параметрах
model_3=LogisticRegression(random_state=random_state)
model_3.fit(features_train,target_train)
#предсказание для валидационной выборки
model_3_predictions_valid=model_3.predict(features_valid)
#подсчет точности предсказаиий
accuracy_score(model_3_predictions_valid,target_valid)

0.7387247278382582

Как видно, точность предсказаний для данного метода составляет примерно 0.74 при стандартных параметрах модели. Добавим гиперпараметры и посмотрим на изменение качества предсказаний.

In [14]:
#обучение модели логистической регрессии при измененных параметрах
model_3=LogisticRegression(random_state=random_state, solver='lbfgs', max_iter=10000)
model_3.fit(features_train,target_train)
#предсказание для валидационной выборки
model_3_predictions_valid=model_3.predict(features_valid)
#подсчет точности предсказаиий
accuracy_score(model_3_predictions_valid,target_valid)

0.7387247278382582

Как видно, видимо для модели логистической регрессии максимальное значение точности предсказаний на тренировочных данных имеет значение 0.74. Попробуем проверить: может ли это значение быть хуже?

In [15]:
#обучение модели логистической регрессии при измененных параметрах
model_3=LogisticRegression(random_state=random_state, solver='lbfgs', max_iter=1)
model_3.fit(features_train,target_train)
#предсказание для валидационной выборки
model_3_predictions_valid=model_3.predict(features_valid)
#подсчет точности предсказаиий
accuracy_score(model_3_predictions_valid,target_valid)

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6936236391912908

Как видно, для значения с одной итерацией качество предсказаний становится меньше, а также выскакивает предупреждение о том, что расчет остановился по причине достижения максимального числа итераций (при этом видимо сообщается о том, что это не максимальная точность, которую мог быть дать данный метод). Теперь попробуем поменять параметр solver:

In [16]:
#обучение модели логистической регрессии при различных методах расчета

for i in ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']:    
    model_3=LogisticRegression(random_state=12345, solver=i)
    model_3.fit(features_train,target_train)
    model_3_predictions=model_3.predict(features_valid)
    print(i,':',model_3.score(features_valid,target_valid))

lbfgs : 0.7387247278382582
liblinear : 0.71850699844479
newton-cg : 0.7387247278382582
sag : 0.6936236391912908
saga : 0.6936236391912908


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


<b>Обновлённый вывод:</b> Из полученных результатов заметно, что для различных методов расчета логистической регрессии получаются различные значения точности. Лучше всех себя показали lbfgs и newton-cg. Также соит отметить, что для sag и saga выпадает предупреждение о несхождении расчёта при стандартном кол-ве итераций</div>

 <b>Обновлённый вывод:</b> Было произведено обучение трех моделей: по методу Дерева решений, Случайного леса и Логистической регрессии. При проверке на точность на валидационных данных первые два метода показали точность около 0.8 (для метода Дерева решений при параметре глубины дерева в 8), а методом Логистической регрессии максимальная точность составила порядка 0.74. В следующем пункте проверим модели на точность на тестовой выборке.</div>

## Проверка моделей на тестовой выборке

In [17]:
%%time
#Проверка модели случайного леса на тестовой выборке
model_2_predictions_test=model_2.predict(features_test)
accuracy_score(model_2_predictions_test,target_test)

CPU times: total: 78.1 ms
Wall time: 88.9 ms


0.8118195956454122

 <b>Обновлённый вывод:</b> При проверке наилучшей модели согласно п.3.1 Случайнго леса было получено значение точности предсказаний в 0.81.</div>

## (бонус) Проверка модели на адекватность

Для проверки на адекватность создадим константную модель с одним значением: тем, которое чаще встречается.

In [18]:
#определим среднее значение: если больше 0.5 - значит 1 больше, чем 0, и наоборот
df['is_ultra'].mean()

0.30647168637212197

Среднее значение по столбцу меньше 0.5. Значит есть смысл заполнить констатную модель значением "0"

In [19]:
#создаём столбец нулей размером с размером столбца target_valid
target_constant=pd.Series([0]*len(target_valid))

#расчет точности предсказания константной модели
accuracy_score(target_constant,target_valid)

0.6936236391912908

Как видно, значение точности предсказания константной модели - 0.7, тогда как значение точности предсказаний методами Дерева решений и Случайного леса - больше. При этом стоит иметь в виду, что модель Логистической регрессии с указанными параметрами неприменима, и её применение необходимо дополнительно докрутить изменением параметров на других выборках для уверенного применения.

In [20]:
#создаём константную модель
dummy_clf = DummyClassifier(strategy="most_frequent")
#наполняем константную модель
dummy_clf.fit(features_train,target_train)
dummy_clf.predict(features_valid)
dummy_clf.score(features_valid,target_valid)

0.6936236391912908

# Общий вывод

По результатам исследования предсказания нужно ли переходить на новый тариф или нет по имеющимся данным получено следующее:

    Лучше всех на валидационной выборке (по соотношению точность/время) показала себя модель Дерева решений с глубиной дерева 8. Также стоит отметить, что почти равную Дереву решений точность показал метод Случайного леса (но со временем работы в 2 раза больше).
    Модель Логистической регрессии не проходит в сравнении тестовой модели с константной моделью, но проходит для валидационной выборки, а значит применяя данный метод необходимо очень внимательно проверять результаты и дополнительно корректировать гиперпараметры.